В работе описана модель Transformer-XL ("extra long") - модификация архитектуры и способа обучения [трансформера]($Attention Is All You Need$), которая позволяет работать с более длинными текстами (или другими последовательностями).

### Long-range dependencies

В задачах NLP под понятием [long-range dependencies](https://en.wikipedia.org/wiki/Long-range_dependence) (дальние зависимости) понимаются смысловые связи между участками текста, далеко отстоящими друг от друга. Иногда важно наделить модель способностью улавливать такие связи. Например, в задаче суммаризации модель обучается составлять краткий пересказ текста. Если дойдя до 1000-го слова модель "забудет" то, что было в начале, то ей вероятно не удастся составить качественный пересказ.

Рекуррентные сети (такие как LSTM) плохо справляются с такой задачей. Например, в [данной работе]($Sharp Nearby, Fuzzy Far Away: How Neural Language Models Use Context$) сети LSTM подается на вход длинный текст и изучается, как изменение, перестановка и удаление слов в начале этого текста влияет на точность ответа модели (точнее, на [перплексию](https://towardsdatascience.com/perplexity-in-language-models-87a196019a94), то есть экспоненту от кроссэнтропии на тестовом датасете). Выясняется, что в LSTM на ответ влияют только последние ~200 слов в тексте, при этом порядок слов влияет на ответ только в последних ~50 словах. Судя по всему, данная проблема обусловлена большой глубиной LSTM-сетей, то есть длиной пути в графе вычислений от входных к выходным данным. Эта же проблема приводит к [затухающим и взрывающимся градиентам]($Gradient Flow in Recurrent Nets: the Difficulty of Learning Long-Term Dependencies$) при обучении.

В механизме self-attention в трансформере токены обмениваются информацией независимо от расстояния между ними, что может облегчить обучение дальним зависимостям. Однако в процессе обучения языковых моделей-трансформеров (см. [GPT и BERT]($GPT и BERT$)) длина входной последовательности обычно фиксирована, и увеличение ее ведет к квадратичному росту объема вычислений и потребляемой памяти.

Обучив языковую модель-трансформер на фрагментах текстов длиной, например, 512, можно делать инференс на текстах произвольной длины. Однако позиционные эмбеддинги для элементов начиная с 512-го никогда не встречались при обучении, и модель может быть к ним не приспособлена (об этом см. также в [этой статье]($Train Short, Test Long: Attention with Linear Biases Enables Input Length Extrapolation$)).

### Transformer-XL

Пусть мы обучаем трансформер на фрагментах длиной $N$ (обычно $N=512$), взятых их текстов существенно большей длины. Если текст имеет длину $10N$, то его можно разбить на $10$ фрагментов. Подход авторов данной статьи заключается в том, что при обработке $t$-го фрагмента $n$-й слой self-attention принимает на вектора $(n-1)$-го слоя как из $t$-го фрагмента, так и из $(t-1)$-го фрагмента.

Рассмотрим этот механизм более подробно. Слой трансформера принимает и возвращает последовательность векторов равной длины. Обозначим за $h_t^n$ массив из $N$ векторов, которые подаются на вход $n$-ному слою при обработке $t$-го фрагмента. В обычном трансформере каждый фрагмент обрабатывается полностью независимо, и слой работает таким образом:

$h_t^{n+1} = \text{TransformerLayer}(h_t^n)$

В Transformer-XL слой трансформера обрабатывает вектора, взятые из предыдущего слоя текущего и предыдущего фрагментов:

$h_t^{n+1} = \text{TransformerLayer}(\text{Concat}(\text{StopGradient}(h_{t-1}^n), h_t^n))[N\text{:}]$

Здесь операция $\text{Concat}$ склеивает вектора из текущего и предыдущего фрагментов, получая $2N$ векторов. Далее слой $\text{TransformerLayer}$ принимает и возвращает $2N$ векторов. Операция $[N\text{:}]$, записанная python-нотацией, означает взятие последних $N$ векторов. При этом градиенты между фрагментами не распространяются, что обозначено операцией $\text{StopGradient}$.

<img src="assets/transformerxl.jpg" width="700" align="center">

Не распространение градиентов между фрагментами означает, что можно сначала делать прямой и обратный проход по первому фрагменту, затем по второму и так далее. В операции self-attention в слое трансформера каждый вектор играет роль запроса, ключа и значения (key, query, value). Выходные вектора, соответствующие входным векторам предыдущего фрагмента, отбрасываются, поэтому можно считать, что запросами являются только вектора, пришедшие из текущего фрагмента, а ключами и значениями - вектора из обоих фрагментов. Это означает, что вектор-эмбеддинг каждого токена текущего фрагмента обновляется под действием векторов токенов текущего и предыдущего фрагментов.

Авторы называют архитектуру рекурентной с той оговоркой, что в данном случае рекуррентная зависимость имеется между текущим слоем текущего фрагмента и предыдущим слоем предыдущего фрагмента. Имея $L$ слоев, модель может заглянуть на $L$ фрагментов назад для получения ответа на текущем фрагменте. Авторы проводят аналогию между своим подходом и методом [truncated backpropagation through time]($Recurrent neural network based language model$) (2010).

В целом данная структура связей между слоями и фрагментами напоминает структуру связей в трансформере, если маска self-attention является [нижней двудиагональной](https://en.wikipedia.org/wiki/Bidiagonal_matrix) матрицей. Поэтому архитектуру Transformer-XL можно описать как "трансформер в трансформере".

<img src="assets/transformerxl2.jpg" width="450" align="center">